# Notebook 7 - Datasets & Evals

<div>
<center><img src="./images/shopping_agent.png" width="600"/></center>
</div>

Welcome to the last exercise! In this exercise, you will add tracing to our **Shopping Assistant**. You will learn how to create datasets on LangSmith and how to run evaluations (evals) to assess your agent's performance systematically.

### Objective:
The goal of this exercise is to:
1. **Set up LangSmith tracing** to monitor and debug your agent's execution in real-time
2. **Create comprehensive datasets** using multiple methods (programmatic, CSV, DataFrame)
3. **Design evaluation tasks** to measure your agent's accuracy and behavior
4. **Run automated evaluations** to validate both final outputs and intermediate steps
5. **Analyze results** in the LangSmith UI to identify areas for improvement

### Why This Exercise?
This exercise is crucial because:
1. **Visibility**: Tracing gives you deep insights into how your agent makes decisions, which tools it calls, and where issues occur
2. **Quality Assurance**: Systematic evaluation ensures your agent performs consistently across various scenarios
3. **Regression Testing**: Datasets allow you to catch bugs when making changes to your agent
4. **Performance Metrics**: Evaluations provide quantifiable metrics to track improvements over time
5. **Production Readiness**: Understanding evaluation patterns prepares you to deploy reliable agents in real-world applications

By the end of this exercise, you'll have hands-on experience creating robust evaluation frameworks that ensure your AI agents work reliably and consistently. You'll be able to catch bugs early, measure improvements, and confidently deploy your agents knowing they've been thoroughly tested.

### Ready to Get Started?
Let's begin by setting up the environment and building a production-ready evaluation pipeline for your AI agents!


## 0. Preparing the Environment

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

load_dotenv(override=True)

## 1. Initializing the Large Language Model (LLM)

In [ ]:
MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"
MODEL_TEMPERATURE = 0

llm = init_chat_model(model=MODEL_NAME, temperature=MODEL_TEMPERATURE)

## 2. Defining the Tools

To build the **Shopping Assistant AI agent**, we'll need two specific tools. While we’ll provide the tool for retrieving clothing data, you’ll get the opportunity to define the calculator tool (or use a prebuilt version).

#### Tools Overview:

1. **Clothing Data Retrieval Tool** (Provided)
   - This tool retrieves basic information about available clothing items, such as type, price, and discount.
   - The agent will use it to respond to queries like:
     - *"What’s the price of a red t-shirt?"*
     - *"Do you have black shoes?"*

2. **Calculator Tool** (To Be Defined)
   - The agent will use this tool to perform mathematical operations, such as:
     - Calculating totals for multiple items.
     - Applying discounts.
   - You can define your own tool (recomended) or use a prebuilt one.

Let’s start by defining the data and the tools.

In [ ]:
import enum

class ProductNames(enum.Enum):
    RedShirt = "red shirt"
    BlueJeans = "blue jeans"
    BlackShoes = "black shoes"
    WhiteSocks = "white socks"
    GrayHat = "gray hat"
    GreenScarf = "green scarf"
    YellowBelt = "yellow belt"
    PurpleGloves = "purple gloves"
    BrownJacket = "brown jacket"
    PinkSweater = "pink sweater"

PRODUCTS = [
    {
        "name": ProductNames.RedShirt.value,
        "price": 20.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish red shirt made from high-quality cotton.",
    },
    {
        "name": ProductNames.BlueJeans.value,
        "price": 40.00,
        "discount": 0.20,
        "currency": "EUR",
        "description": "A pair of comfortable blue jeans made from durable denim.",
    },
    {
        "name": ProductNames.BlackShoes.value,
        "price": 30.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pair of elegant black shoes made from genuine leather.",
    },
    {
        "name": ProductNames.WhiteSocks.value,
        "price": 5.00,
        "discount": 0.05,
        "currency": "EUR",
        "description": "A pack of soft white socks made from breathable cotton.",
    },
    {
        "name": ProductNames.GrayHat.value,
        "price": 10.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A stylish gray hat made from lightweight fabric.",
    },
    {
        "name": ProductNames.GreenScarf.value,
        "price": 15.00,
        "discount": 0.12,
        "currency": "EUR",
        "description": "A warm green scarf made from wool.",
    },
    {
        "name": ProductNames.YellowBelt.value,
        "price": 8.00,
        "discount": 0.08,
        "currency": "EUR",
        "description": "A yellow belt made from synthetic leather.",
    },
    {
        "name": ProductNames.PurpleGloves.value,
        "price": 12.00,
        "discount": 0.10,
        "currency": "EUR",
        "description": "A pair of purple gloves made from soft fabric.",
    },
    {
        "name": ProductNames.BrownJacket.value,
        "price": 50.00,
        "discount": 0.25,
        "currency": "EUR",
        "description": "A brown jacket made from genuine leather.",
    },
    {
        "name": ProductNames.PinkSweater.value,
        "price": 25.00,
        "discount": 0.15,
        "currency": "EUR",
        "description": "A pink sweater made from high-quality cotton.",
    }
]

In [ ]:
from tools.math import prebuilt_calculator

@tool
def get_product_info(product_name: ProductNames) -> dict:
    """
    This tool retrieves information about a product based on its name.
    The input is a product name from the ProductNames enum.
    The output is a dictionary containing the product's details such as price, discount, currency, and description.
    If the product is not found, it returns None.
    """
    for product in PRODUCTS:
        if product["name"] == product_name.value:
            return product
    return None

@tool
def calculator(expression: str) -> float:
    """
    This tool evaluates a mathematical expression provided as a string and returns the result as a float.
    The input is a string representing a mathematical expression (e.g., "2 + 2 * 3").
    The output is the result of the evaluated expression as a float.
    If the expression is invalid or cannot be evaluated, it raises an appropriate error.
    """
    return prebuilt_calculator(expression)

Test both tools to ensure they work correctly before integrating them into the agent.

In [ ]:
get_product_info.invoke({"product_name": ProductNames.RedShirt})

In [ ]:
calculator.invoke({"expression": "2 + 2 * 10"})

## 3. Crafting the Agent System Prompt

Define a system prompt for your agent. The agent should:
1. Help customers browse a clothing shop by retrieving details about items (using the `get_clothing_info` tool).
2. Perform calculations, such as computing totals or applying discounts, with the `perform_calculation` tool.
3. Politely decline irrelevant or unsupported queries (e.g., unrelated to clothing or shopping).

In [ ]:
SHOPPING_AGENT_PROMPT = """"
You are a helpful shopping assistant for a clothing store. You help customers find products, provide information about them, calculate prices, discount, and assist with their shopping needs.

Tone of voice: friendly, professional, and approachable.
Language: Match the user's language.

Tools you can use:
- get_product_info: Use this tool to get detailed information about a product by providing its name from the ProductNames enum.
- calculator: Use this tool to perform mathematical calculations.

Duties:
- Understand the user's shopping needs and preferences.
- Provide accurate and detailed information about products.
- Compute prices, discounts, and totals as needed.
- Correctly apply discounts to product prices: final_price = price * (1 - discount) using the calculator tool.
- Always think step-by-step and use the tools when necessary to ensure accuracy.
- Do not rely on memory; use the tools to fetch information as needed.
- Consider the discount only when explicitly asked by the user.
"""

## 4. Building the AI Agent

Now that you have the tools and system prompt ready, it’s time to build the AI agent. The agent should have access to both tools and should also have short-term memory to keep track of the conversation context.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4

# Initialize MemorySaver
memory = MemorySaver()

config = {
    "configurable": {
        "thread_id": str(uuid4())
        }
}

graph = create_react_agent(
    model=llm,
    tools=[get_product_info, calculator],
    prompt=SHOPPING_AGENT_PROMPT,
    checkpointer=memory,
).with_config(config)

## 5. Dataset Creation and Evaluations

Now that we have a working Shopping Assistant agent, it's time to ensure it works reliably and consistently. In this section, we'll setup an evaluation with a simple dataset.

### Why This Matters:
Evaluations allows you to:
- **Catch regressions** when modifying prompts or tools
- **Quantify improvements** with concrete metrics
- **Build confidence** before deploying to production
- **Understand failures** through detailed trace analysis

> **Note**: Ensure your `.env` file contains valid `LANGSMITH_API_KEY` and `LANGSMITH_PROJECT` values. This section assumes LangSmith is properly configured and won't display sensitive credentials.

### Dataset Creation
Let's start by creating our test dataset!

In [ ]:
# 📦 Crea (o recupera) un dataset ad hoc per l'agente
from langsmith import Client

ls_client = Client()

DATASET_NAME = "shopping-agent-dataset-v3"
DATASET_DESC = "Domande su prodotti e richieste di totali per l'agente con get_product_info + calculator."

examples = [
    {
        "inputs": {
            "question": "How much does the red shirt cost normally?",
            "expected_tools": ["get_product_info"]
        },
        "outputs": {
            "answer": "20"
        },
    },
    {
        "inputs": {
            "question": "How much does the red shirt cost considering the discount?",
            "expected_tools": ["get_product_info", "calculator"]
        },
        "outputs": {
            "answer": "18"
        },
    },
    {
        "inputs": {
            "question": "What's the total price for 1 pair of black shoes and 1 pack of white socks considering the discount?",
            "expected_tools": ["get_product_info", "calculator"]
        },
        "outputs": {
            "answer": "30.25"
        },
    },
    {
        "inputs": {
            "question": "Price of the blue jeans?",
            "expected_tools": ["get_product_info"]
        },
        "outputs": {
            "answer": "40"
        },
    },
    {
        "inputs": {
            "question": "I want 2 blue jeans and 1 red shirt. What's the total considering the discount?",
            "expected_tools": ["get_product_info", "calculator"]
        },
        "outputs": {
            "answer": "82"
        },
    },
]

try:
    dataset = ls_client.read_dataset(DATASET_NAME)
except Exception:
    dataset = ls_client.create_dataset(dataset_name=DATASET_NAME, description=DATASET_DESC)

ls_client.create_examples(dataset_id=dataset.id, examples=examples)

print(f"Dataset pronto: {dataset.name} (id={dataset.id}) — esempi caricati: {len(examples)}")

In the cell below, we define a function to convert each dataset example into the agent's input state and create a runnable pipeline (`target`) that processes these examples through the agent. This prepares the workflow for systematic evaluation of the agent's responses.

In [ ]:
from typing import Dict

def example_to_state(inputs: Dict) -> Dict:
    q = inputs["question"]
    return {"messages": [("user", q)]}

target = example_to_state | graph
print("Target pronto (example_to_state | graph).")


### Defining Evaluation Functions and Criteria

In the following section, we define custom evaluation functions—our criteria—for systematically assessing the Shopping Assistant agent's performance. Each function targets a specific aspect of the agent's behavior:

- **price_correct**: Checks if the agent's final answer contains the correct price or total, comparing the first number in its output to the expected value with a small tolerance for floating-point precision.

- **right_tools**: Verifies that the agent used all the tools expected for each example (e.g., product info retrieval, calculator), ensuring the reasoning process is traceable and correct.

- **calc_arguments_sane**: For questions requiring a total or sum, this criterion ensures that the calculator tool was called with an expression containing at least two numbers, confirming that the agent performed a meaningful calculation rather than a trivial or incorrect operation.

These criteria help guarantee that the agent not only produces accurate answers, but also follows the correct reasoning steps and tool usage throughout its workflow.

In [ ]:
import re
from typing import Dict

NUM_TOLERANCE = 1e-6

def extract_first_number(text: str):
    if text is None:
        return None
    m = re.search(r"-?\d+(?:\.\d+)?", str(text))
    return float(m.group(0)) if m else None

def price_correct(outputs: Dict, reference_outputs: Dict) -> bool:
    try:
        final_msg = outputs["messages"][-1]
        actual_text = getattr(final_msg, "content", final_msg)
    except Exception:
        actual_text = str(outputs)
    actual_num = extract_first_number(actual_text)
    expected_num = extract_first_number(reference_outputs.get("answer"))
    if actual_num is None or expected_num is None:
        return False
    return abs(actual_num - expected_num) <= NUM_TOLERANCE


In [ ]:
from typing import Dict, List
from langsmith.schemas import Example
import re

def _collect_tool_calls_from_messages(outputs: Dict) -> List[Dict]:
    """
    Extracts all tool_calls from the messages in the graph output (if available).
    """
    tool_calls = []
    try:
        for msg in outputs["messages"]:
            tc = getattr(msg, "tool_calls", None) or getattr(msg, "additional_kwargs", {}).get("tool_calls")
            if tc:
                if isinstance(tc, dict):
                    tc = [tc]
                tool_calls.extend(tc)
    except Exception:
        pass
    return tool_calls

def right_tools(outputs: Dict, example: Example) -> Dict:
    """
    Check if the expected tools were used in the execution.
    """
    expected = set(example.inputs.get("expected_tools", []))
    if not expected:
        return {"key": "right_tools", "value": True, "comment": "No tools expected."}
    used = {(tc.get("name") if isinstance(tc, dict) else getattr(tc, "name", None)) for tc in _collect_tool_calls_from_messages(outputs)}
    ok = expected.issubset(used)
    missing = expected - used
    return {"key": "right_tools", "value": bool(ok), "comment": f"Missing: {sorted(missing)}" if missing else ""}

def calc_arguments_sane(outputs: Dict, example: Example) -> Dict:
    """
    If the prompt asks for a total, check that the argument to the calculator contains at least 2 numbers.
    """
    q = example.inputs.get("question", "").lower()
    if "total" not in q and "sum" not in q and "price for" not in q:
        return {"key": "calc_args_sane", "value": True}
    tool_calls = _collect_tool_calls_from_messages(outputs)
    calc_calls = [tc for tc in tool_calls if (tc.get("name") if isinstance(tc, dict) else getattr(tc, "name", "")) == "calculator"]
    if not calc_calls:
        return {"key": "calc_args_sane", "value": False, "comment": "No calculator calls found."}
    def _extract_expr(tc):
        args = tc.get("args") if isinstance(tc, dict) else getattr(tc, "args", {})
        if not args and isinstance(tc, dict):
            args = tc.get("arguments") or {}
        return (args or {}).get("expression")
    exprs = [_extract_expr(tc) for tc in calc_calls]
    has_enough_numbers = any(len(re.findall(r"-?\d+(?:\.\d+)?", str(e or ""))) >= 2 for e in exprs)
    return {"key": "calc_args_sane", "value": bool(has_enough_numbers), "comment": f"exprs seen={exprs}"}

### Running the Experiment with Custom Evaluation Criteria

In the next cell, we define and launch our experiment using the custom evaluation criteria described above. This step systematically tests the Shopping Assistant agent against our dataset, applying checks for answer correctness, tool usage, and calculation logic. Once the experiment completes, you can review detailed results and traces directly in the LangSmith UI to analyze performance and identify areas for improvement.

In [ ]:
import datetime
from langsmith import aevaluate

EXPERIMENT_PREFIX = f"agent-evals-{datetime.date.today()}"
print("Running evaluation… this will create an Experiment in LangSmith UI.")

experiment_results = await aevaluate(
    target,
    data=DATASET_NAME,
    evaluators=[price_correct, right_tools, calc_arguments_sane],
    experiment_prefix=EXPERIMENT_PREFIX,
    max_concurrency=4,
)
print("Done. Controlla LangSmith → Experiments per i risultati.")


## Extra
### Create a dataset from a CSV file

In this section, we will demonstrate how you can create a dataset by uploading a CSV file.
First, ensure your CSV file is properly formatted with columns that represent your input and output keys. These keys will be utilized to map your data properly during the upload. You can specify an optional name and description for your dataset. Otherwise, the file name will be used as the dataset name and no description will be provided.

In [ ]:
from langsmith import Client

client = Client()
csv_file = 'data/sample_dataset.csv'  
input_keys = ['question', 'expected_tools'] 
output_keys = ['answer'] 

dataset = client.upload_csv(
  csv_file=csv_file,
  input_keys=input_keys,
  output_keys=output_keys,
  name="Shopping Agent CSV Dataset",
  description="Dataset created from a CSV file with shopping agent questions",
  data_type="kv"
)

### Create a dataset from pandas DataFrame (Python only)
The python client offers an additional convenience method to upload a dataset from a pandas dataframe.

In [ ]:
from langsmith import Client
import pandas as pd

client = Client()
df = pd.read_parquet('data/sample_dataset.parquet') 
input_keys = ['question', 'expected_tools']  
output_keys = ['answer'] 

dataset = client.upload_dataframe(
    df=df,
    input_keys=input_keys,
    output_keys=output_keys,
    name="Shopping Agent Parquet Dataset",
    description="Dataset created from a parquet file with shopping agent questions",
    data_type="kv"
)

### Fetch datasets
You can programmatically fetch datasets from LangSmith using the list_datasets/listDatasets method in the Python and TypeScript SDKs. Below are some common calls.

In [ ]:
from langsmith import Client

client = Client()

In [ ]:
datasets = client.list_datasets()

for d in datasets:
    print(f"{d.name} (id={d.id}) - {d.description} - {d.created_at}")

### List datasets by name
If you want to search by the exact name, you can do the following:

In [ ]:
datasets = client.list_datasets(dataset_name="Shopping Agent CSV Dataset")

for d in datasets:
    print(f"{d.name} (id={d.id}) - {d.description} - {d.created_at}")